# Aufbau einer Pipeline zur Vorverarbeitung der Daten <div class="tocSkip">


In diesem Notebook erfolgt die Aufbereitung der Daten

## Hilfsfunktionen und Einstellungen <div class="tocSkip">


In [5]:
%run "../lib/settings.py"

%reload_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

# to print output of all statements and not just the last
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import textacy.preprocessing as tprep
import textacy


In [6]:
def txa_pd_read_sql(database, sql):
    """
    Open connection to sqlite database and return result 
    of sql query as pandas datafram
    :param sql: query
    :return: dataframe
    """
    with sqlite3.connect(database) as con:
        cur = con.cursor()
        df = pd.read_sql_query(sql, con, )
        return df
    
    
def convert_list_to_string(org_list, seperator=' '):
    """ Convert list to string, by joining all item in list with given separator.
        Returns the concatenated string """
    return seperator.join(org_list)

def filtering(liste):
    ergebnis= [t for t in liste if t not in stopwords]
    ergebnis = " ".join(ergebnis)
    return ergebnis

## Initialisierung


In [7]:
df = txa_pd_read_sql('../resources/dbt_speeches.sqlite','SELECT * FROM speeches')

In [8]:
df.head(4)

,speech_id,speaker,affiliation,text,session_id,session_date,session_start,session_end,legislative_period
0,ID194300100,Olaf Scholz,Bundesminister BMF,Herr Präsident! Meine Damen und Herren! Griechenland wird das dritte Hilfsprogramm planmäßig am 20. August 2018 beenden. Damit kann Griechenland die Rettungsschirme nach acht Jahren verlassen und ...,43,29.06.2018,09:00,17:07,19
1,ID194300200,Peter Boehringer,AfD,Herr Präsident! Liebe Kolleginnen! Liebe Kollegen! In der heutigen Vorlage steht auf 480 Seiten eigentlich nur eines: Griechenland bekommt eine weitere Finanzspritze in Milliardenhöhe. Konkret: De...,43,29.06.2018,09:00,17:07,19
2,ID194300300,Eckhardt Rehberg,CDU/CSU,"Herr Präsident! Liebe Kolleginnen und Kollegen! Wenn ich manche Rede zu Griechenland in den letzten acht Jahren Revue passieren lasse, dann fallen mir auch viele Aussagen zu Irland, Portugal, Span...",43,29.06.2018,09:00,17:07,19
3,ID194300400,Christian Dürr,FDP,"Herr Präsident! Meine sehr verehrten Kolleginnen und Kollegen! Lieber Kollege Rehberg, Sie sagen: Wir sind im dritten Programm, nicht im vierten. – Das ist ja keine Fernsehsendung hier. In der Nac...",43,29.06.2018,09:00,17:07,19


## Berechnung der Reinheit der Daten 

In [9]:
RE_SUSPICIOUS = re.compile(r'[&#<>{}\()[\]\\^^$%@]')
text = df['text']
def impurity(text, min_len=10):
    """returns the share of suspicious characters in a text"""
    if text == None or len(text) < min_len:
        return 0
    else:
        return len(RE_SUSPICIOUS.findall(text))/len(text)

print(f'About {round(impurity(text.all()) *100,2)}% of the characters are impure')

About 0% of the characters are impure


=> Wir haben hier realtiv saubere Daten<div class="tocSkip">


In [10]:
import spacy
from spacy.tokenizer import Tokenizer
from spacy.util import compile_prefix_regex, \
                       compile_infix_regex, compile_suffix_regex

nlp = spacy.load('de_core_news_md', disable=['parser'])


In [11]:
def display_nlp(doc, include_punct=False):
    """Generate data frame for visualization of spaCy tokens."""
    rows = []
    for i, t in enumerate(doc):
        if not t.is_punct or include_punct:
            row = {'token': i,  'text': t.text, 'lemma_': t.lemma_,
                   'is_stop': t.is_stop, 'is_alpha': t.is_alpha,
                   'pos_': t.pos_, 'dep_': t.dep_,
                   'ent_type_': t.ent_type_, 'ent_iob_': t.ent_iob_}
            rows.append(row)
    df = pd.DataFrame(rows).set_index('token')
    df.index.name = None
    return df

## Funktion zur Tokenisierung

In [12]:
def custom_tokenizer(nlp):

    # use default patterns except the ones matched by re.search
    prefixes = [pattern for pattern in nlp.Defaults.prefixes
                if pattern not in ['\(']]
    suffixes = [pattern for pattern in nlp.Defaults.suffixes
                if pattern not in ['\)']]
    infixes  = [pattern for pattern in nlp.Defaults.infixes
                if not re.search(pattern, 'xx-xx')]

    return Tokenizer(vocab          = nlp.vocab,
                     rules          = nlp.Defaults.tokenizer_exceptions,
                     prefix_search  = compile_prefix_regex(prefixes).search,
                     suffix_search  = compile_suffix_regex(suffixes).search,
                     infix_finditer = compile_infix_regex(infixes).finditer,
                     token_match    = nlp.Defaults.token_match)



In [13]:
def extract_noun_phrases(doc, preceding_pos=['NOUN'], sep='_'):
    patterns = []
    for pos in preceding_pos:
        patterns.append(f"POS:{pos} POS:NOUN:+")
    spans = textacy.extract.matches(doc, patterns=patterns)
    return [sep.join([t.lemma_ for t in s]) for s in spans]



**Wir haben eine eigene Stopword-List definiert, deren Verwendung in den meisten Reden auftretende und bedeutungsarme Wörter entfernt.**

In [16]:
with open('../resources/stopwords_de.txt', encoding='utf-8') as file:
    stopwords = file.read().splitlines()

In [17]:
from tqdm import tqdm
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import numpy as np
import pandas as pd 
pd.set_option('display.max_colwidth', -1)
import spacy
from spacy import displacy


def extract_entities(doc, include_types=None, sep='_'):

    ents = textacy.extract.entities(doc, 
             include_types=include_types, 
             exclude_types=None, 
             drop_determiners=True, 
             min_freq=1)
    ents = [e for e in ents if len(e) <= 2]
    ents = [sep.join([t.text for t in e]).strip('.\n')+'/'+e.label_ for e in ents]
    return ents



def extract_nlp(doc):
    nlp.tokenizer = custom_tokenizer(nlp)
    return {
    'lemmas'          : extract_lemmas(doc,exclude_pos = ['PART', 'PUNCT', 'DET', 'PRON', 'SYM', 'SPACE'],filter_stops = True),
    'adjs_verbs'      : extract_lemmas(doc, include_pos = ['ADJ', 'VERB']),
    'nouns'           : extract_lemmas(doc, include_pos = ['NOUN', 'PROPN']),
    'noun_phrases'    : extract_noun_phrases(doc, ['NOUN']),
    'adj_noun_phrases': extract_noun_phrases(doc, ['ADJ']),
    'entities'        : extract_entities(doc, ['PER', 'ORG', 'GPE', 'LOC'])
    }


def extract_lemmas(doc, **kwargs):
    return [t.lemma_ for t in textacy.extract.words(doc, **kwargs)]



In [18]:
df.head(1)

speech_id      speaker         affiliation  \
0  ID194300100  Olaf Scholz  Bundesminister BMF   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [19]:
nlp_columns = list(extract_nlp(nlp.make_doc('')).keys())
print(nlp_columns)

for col in nlp_columns:
    df[col] = None

['lemmas', 'adjs_verbs', 'nouns', 'noun_phrases', 'adj_noun_phrases', 'entities']


**Vorsicht: Die Berechnung dauert mindestens 40 Mintuen**

In [20]:
batch_size = 100
num_batches = math.ceil(len(df) / batch_size)

for i in tqdm(range(0, len(df), batch_size), total=num_batches):
    
    # spaCy Batch-Verarbeitung mit nlp.pipe, liefert eine Liste von Ergebnis-Docs
    docs = nlp.pipe(df['text'][i:i+batch_size])
    
    # Extraktion der Lemmas und Eintragen im DataFrame für einen Batch
    for j, doc in enumerate(docs):
        for col, values in extract_nlp(doc).items():
            df[col].iloc[i+j] = values
            
for column in nlp_columns:
    df[column] = df[column].progress_map(lambda tokens: " ".join(tokens))
    
df[nlp_columns].head(10)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 247/247 [35:12<00:00,  8.55s/it]


  0%|          | 0/24647 [00:00<?, ?it/s]

  0%|          | 0/24647 [00:00<?, ?it/s]

  0%|          | 0/24647 [00:00<?, ?it/s]

  0%|          | 0/24647 [00:00<?, ?it/s]

  0%|          | 0/24647 [00:00<?, ?it/s]

  0%|          | 0/24647 [00:00<?, ?it/s]

lemmas  \
0  Herr Präsident Dame Herr Griechenland Hilfsprogramm planmäßig 20. August 2018 beenden Griechenland Rettungsschirme verlassen Chance Bein stehen Nachricht Nachricht anschließen Europäische Union Euro-Gruppe gemeinsam gelingen Hilfsmaßnahmen Portugal Irland finanziell Schwierigkeit helfen europäische Nation gewissermaßen Ergebnis europäisch Solidarität Hilfsnotwendigkeiten herausgekommen Bein stehen glauben zeigen Europa Stärke Kooperation gemeinsam Herausforderung Zukunft bewältigen Europa Kooperation diskutieren Grund ausdrücklich sagen Staatschefs Europäische Union gestern Nacht gemeinsam Position Frage Asyl- Migrationspolitik finden Fortschritt glauben Grundlage Herausforderung Europa Umgang Fluchtmigration Erfolg Euro-Gruppe vereinbarte Paket vorstellen umfasst breit Unterstützung ausdrücklich bitten deutschen Bundestag Maßnahme beantragen unterstützen freuen Debatte geschehen unterhalten Entwicklung Griechenland letzt zutragen genau hinschauen stellen fest Reform unternehmen miteinander feststellen Griechenland gut Weg Anstrengung Regierung Parlamentes Bürgerin Bürger Griechenland nehmen groß Hilfe leisten Europäer Zurverfügungstellung Fähigkeit Kredit aufnehmen weitergeben Manchmal verwechseln politisch Debatte Direktzahlungen Kredit verleihen Irrtum bleiben wiederholen tatsächlich geschehen Falle Irland Portugal klappen Augenblick Land Zugang Finanzmärkten verlieren sorgen Zugang Hilfe erhalten bleiben Kredit gemeinschaftlich aufnehmen Fall EFSF ESM zurückzahlen Selbstverständlich umfasst lang Zeitraum Deutschland Kredit schwierig Situation aufnehmen zurückzahlen Ding langfristig hören Natur Sache glauben notwendig Schritt letzte Tranche 15 Milliarde Euro bewilligen eigentlich Botschaft 15 Milliarde Euro letzt Tranche keineswegs gesamt Kapazität ESM Griechenland Aussicht stellen ausschöpfen ermöglichen möglicherweise Anspruch nehmen Anspruch nehmen letzte Tranche dienen Übergang Rückkehr Refinanzierung Kapitalmärkte begleiten Genauso Portugal Irland genauso Griechenland funktionieren Klar lang Programm Hilfe groß ergänzen Entscheidung Zukunft wichtig Laufzeitverlängerung Zinsstundung Krediterleichterungen schaffen Aussicht stellen 2016. Herr Bundesminister Scholz Kollege Ehrhorn AfD gerne Zwischenfrage stellen Gestatten Bitte Herr Kollege Herr Minister vorgestern Frage konfrontieren ausführen Bundestag August 2015 Rettungspaket zustimmen einzig wichtig Voraussetzung nämlich IWF Rettungspaket beteiligen Beteiligung geben rede explizit vorgestern Rettungspaket Wahrheit entsprechen vorgestern fragen wiederholen Frage Geschäftsgrundlage Zustimmung Bundestag erlöschen Auszahlung Rettungspaketes befürworten Dank schön Schöne Frage wiederholen gleich Frage stellen wiederholen gleich Antwort geben IWF dabeihaben IWF Kredit Höhe 10 Milliarde Euro laufen 2024 zurückzahlen Programm Rolle spielen IWF Geld kümmern vorstellen überlegen eigentlich Sinn obwohl ESM-Kreditrahmen ausschöpfen IWF 1,6 Milliarde obendrauf geben lassen gewissermaßen Situation wiederholen ausdrücklich IWF Nachprogrammüberwachung politische Zweck Beschluß Haus erreichen Übrigen Punkt kommen hinten Rede weglassen sagen verlängern Zahlungszeiträume helfen Kredittragfähigkeit sorgen Rückkehr Refinanzierung Kapitalmärkte gelingen eingeübt Praxis erreichen verzichten Zinsaufschlag Beschluß 2016 2017 ankündigen Gewinn Zentralbank anfangs gegeben bilateral Darlehen erhalten griechische Politik erreichbar Ding Weg Zustimmung bitten hören vereinbart sprechen kommen Nachprogrammüberwachung geben Anstrengung Bürgerin Bürger Griechenland Regierung unternehmen aufhören vergeblich gerne Politik Griechenland Zukunft entwickeln entsprechen Reformmaßnahmen letzt orientieren Klar Spielräume groß umso groß groß wirtschaftlich Erfolg Land Fall Nachprogrammüberwachung sicherstellen Griechenland Zukunft Reformpolitik verfolgen letzt einschlagen Dame Herr Schluss sagen Zeichen europäisch Solidarität geben erreichen europäisch Union gemeinsam zeigen zusammenhalten Lage Pro

**Filteren der Einträge anhand unserer Word-Liste**

In [21]:
df['lemmas'] = df['lemmas'].apply(str.split).progress_apply(filtering)


  0%|          | 0/24647 [00:00<?, ?it/s]

In [22]:
df.head(3)

speech_id           speaker         affiliation  \
0  ID194300100  Olaf Scholz       Bundesminister BMF   
1  ID194300200  Peter Boehringer  AfD                  
2  ID194300300  Eckhardt Rehberg  CDU/CSU              

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

**Darüber hinaus wurden zahlreiche Bereinigungsschritte händisch durchgeführt, deren Erläuterung das Notebook zu ausführlich machen würde. Deshalb wird davon abgesehen**

**In den kommenden Notebooks wird der Name "clean_text" anstatt "lemmas" verwendet. Das hat aber keinen Einfluss auf den Inhalt der Spalte.**